## **Model Tuning Notebook**

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('../')

from src.models.models import *
from src.utils.modeling import *
from src.utils.preproc import *

import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam

In [4]:
# because the utils in the src are designed to be run from the root of the project,
# and by default jupyter runs from the notebook directory we need to change the working directory to the root

def find_project_root(filename=".git"): # .git is located in the root of the project
    current_dir = os.getcwd()
    while current_dir != os.path.dirname(current_dir): # stops only when at the root (moves up 1 level each iteration)
        if filename in os.listdir(current_dir):
            return current_dir
        current_dir = os.path.dirname(current_dir)

project_root = find_project_root()
os.chdir(project_root)  # change the working directory to the project root

print("Project root:", project_root, "CWD:", os.getcwd())

Project root: d:\deep_learning_project CWD: d:\deep_learning_project


### **Binary Classification Models Fine-Tuning**

#### **Traditional**

#### **Pre-Trained**

**VGG 16**

In [7]:
train_gen, val_gen, X_test, y_test, class_weights = preproc_pipeline(desired_magnification='200X', 
                                                    image_resolution=(224, 224), 
                                                    classification_type='binary',
                                                    csv_path='image_metadata/updated_image_data.csv')

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

